<a href="https://colab.research.google.com/github/DeJezuz/AI-Future-Directions-Assignment/blob/main/models/recyclables_int8_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # Select your recyclables.zip file when prompted

Saving recyclables.zip to recyclables.zip


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import zipfile
import os

# Unzip the uploaded file
with zipfile.ZipFile('recyclables.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/recyclables') # Extract to a directory named 'recyclables'

img_size = (160, 160)
batch_size = 32

# Adjust the directory path to account for the nested folder created by unzipping
base_dir = '/content/recyclables/recyclables'

# Load all images into train_ds and val_ds, as validation_split is problematic for 4 images
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    seed=42, # Keep seed for reproducibility if needed for other operations
    image_size=img_size, batch_size=batch_size
)
# For the purpose of representative dataset for quantization, val_ds can be the same as train_ds given the tiny dataset size.
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    seed=42,
    image_size=img_size, batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)

Found 4 files belonging to 3 classes.
Found 4 files belonging to 3 classes.


### Define a simple CNN model

In [7]:
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(img_size[0], img_size[1], 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 160, 160, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 160, 160, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 80, 80, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 80, 80, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 40, 40, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 40, 40, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,276,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,300,899 (12.59 MB)

 Trainable params: 3,300,899 (12.59 MB)

 Non-trainable params: 0 (0.00 B)

### Compile the model

In [8]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

### Train the model (for demonstration purposes with very few images)

In [9]:
epochs = 5 # Small number of epochs due to very limited data
history = model.fit(
    train_ds,
    epochs=epochs
)

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 1.0728
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.5000 - loss: 5.9215
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.5000 - loss: 1.6502
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.2500 - loss: 2.0216
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.2500 - loss: 1.5210


In [11]:
# Load a lightweight pretrained model
base_model = keras.applications.MobileNetV2(
    input_shape=img_size + (3,),  # (160, 160, 3)
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze the base model

# Add classification layers
inputs = keras.Input(shape=img_size + (3,))
x = keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Compile and train
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_ds, epochs=5)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.2500 - loss: 1.1386
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.5000 - loss: 1.0695
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.5000 - loss: 1.0492
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5000 - loss: 1.0469
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.5000 - loss: 1.0555


In [18]:
# Create a representative dataset for quantization
def representative_data_gen():
    # Ensure val_ds is defined and contains data for this to work
    for images, _ in val_ds.take(50):
        yield [tf.cast(images, tf.float32)]

# Set up the converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert the model
tflite_model = converter.convert()

# Save the model to a file
with open("recyclables_int8.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ File saved: recyclables_int8.tflite")

Saved artifact at '/tmp/tmpk02p7d0n'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='keras_tensor_165')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140446037951504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037952848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037951120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037950928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037953424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037951696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037953040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037953232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037952656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140446037954576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404460379

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ File saved: recyclables_int8.tflite


In [19]:
from google.colab import files
files.download("recyclables_int8.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>